### 데이터 준비

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('./stock data/samsung.csv')
data=data.dropna(axis=0)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-17,36580.0,37460.0,36580.0,36960.0,7811.878418,8150700.0
1,2017-01-18,37040.0,37500.0,36620.0,36940.0,7807.651367,8875400.0
2,2017-01-19,37720.0,37920.0,37020.0,37480.0,7921.786133,9418900.0
3,2017-01-20,37120.0,37420.0,36880.0,37200.0,7862.604004,9185000.0
4,2017-01-23,37200.0,38060.0,37000.0,38060.0,8044.374512,8810400.0


------------------------

### 1. Bollinger band

볼린저 밴드(Boliinger band) : 시리즈의 이동 편균값에 표준편차를 빼고 더한 범위를 의미

- UBB = MA_w(x) + k * sd(x) 

- MBB = MA_w(x)

- LBB = MA_w(x) - k * sd(x)

In [2]:
def bollinger_band(data, w=20, k=2):
        
    mbb = data.rolling(w).mean()
    
    ubb = mbb + k * data.rolling(w).std()
    
    lbb = mbb - k * data.rolling(w).std()
    
    result = np.array([data,mbb,ubb,lbb])
    
    df = pd.DataFrame(result).T
    df.columns = ["close", "mbb", "ubb", "lbb"]
    
    return df

In [3]:
#Example

bollinger_band(data['Close'],10,2)

,close,mbb,ubb,lbb
0,36960.0,NaN,NaN,NaN
1,36940.0,NaN,NaN,NaN
2,37480.0,NaN,NaN,NaN
3,37200.0,NaN,NaN,NaN
4,38060.0,NaN,NaN,NaN
...,...,...,...,...
716,60000.0,56920.0,60488.753284,53351.246716
717,60000.0,57270.0,61310.956982,53229.043018
718,59000.0,57590.0,61620.384597,53559.615403
719,60700.0,58140.0,62221.611882,54058.388118


---

### 2. Moving Average

이동 평균선(Moving Average) : 일정기간 동안의 주가를 산술 평균하 값인 주가이동평균을 연결한 선.

In [4]:
def moving_average(data, w=5):
        
    ma_w=data.ewm(span=w).mean()
    ma_5=data.ewm(span=5).mean()
    ma_10=data.ewm(span=10).mean()
    ma_15=data.ewm(span=15).mean()
    ma_20=data.ewm(span=20).mean()
    ma_30=data.ewm(span=30).mean()
    ma_60=data.ewm(span=60).mean()
    
    result = np.array([data,ma_w,ma_5,ma_10,ma_15,ma_20,ma_30,ma_60])

    df = pd.DataFrame(result).T
    df.columns = ["close", "user_ma_{}".format(w), "ma_5", "ma_10","ma_15","ma_20","ma_30","ma_60" ]
    
    if w == 5:
        del df['user_ma_5']
    
    
    return df

In [5]:
#Example

moving_average(data['Close'], 3)

,close,user_ma_3,ma_5,ma_10,ma_15,ma_20,ma_30,ma_60
0,36960.0,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000
1,36940.0,36946.666667,36948.000000,36949.000000,36949.333333,36949.500000,36949.666667,36949.833333
2,37480.0,37251.428571,37200.000000,37162.458472,37150.295858,37144.296420,37138.356164,37132.478474
3,37200.0,37224.000000,37200.000000,37174.826733,37165.309735,37160.377182,37155.341287,37150.212094
4,38060.0,37655.483871,37530.142180,37428.935903,37394.910093,37377.988286,37361.173047,37344.498887
...,...,...,...,...,...,...,...,...
716,60000.0,59228.401356,58501.075821,57388.920152,56649.522350,56055.399879,55111.799182,53105.594217
717,60000.0,59614.200678,59000.717214,57863.661942,57068.332057,56431.076081,55427.166977,53331.640308
718,59000.0,59307.100339,59000.478143,58070.268862,57309.790550,56675.735502,55657.672333,53517.488167
719,60700.0,60003.550170,59566.985429,58548.401796,57733.566731,57058.998787,55982.983796,53752.980358


---

### 3. MACD

이동평균 수렴 확산지수(MACD; Moving Average Convergence & Divergence) : 이동 편규선끼리의 차이를 통해 주가 흐름을 알아볼때 이용

- MACD = 12일(fast) 지수 이동 평균선 - 26일(slow) 지수이동 평균선

- MACD signal = MACD의 9일(signal) 단순이동편균선

- MACD Oscillator = MACD signal 값을 막대로 표현한 보조지표 

In [6]:
def macd(data, fast=12, slow=26, signal=9) :
    
    fast = data.ewm(span=fast).mean() 
    slow = data.ewm(span=slow).mean() 
    macd = fast - slow 
    signal = macd.ewm(span=signal).mean() 
    osc = macd - signal 
    
    result = np.array([data,macd,fast,slow, signal,osc])
    df = pd.DataFrame(result).T
    df.columns = ["close", "macd", "fast", "slow","signal","osc"]
    
    return df

In [7]:
#Example

macd(data['Close'])

,close,macd,fast,slow,signal,osc
0,36960.0,0.000000,36960.000000,36960.000000,0.000000,0.000000
1,36940.0,-0.448718,36949.166667,36949.615385,-0.249288,-0.199430
2,37480.0,16.173612,37156.351039,37140.177427,6.481409,9.692203
3,37200.0,13.228106,37170.129310,37156.901204,8.766875,4.461231
4,38060.0,45.570456,37411.903996,37366.333540,19.715109,25.855348
...,...,...,...,...,...,...
716,60000.0,1609.271920,57069.504906,55460.232986,1276.665740,332.606180
717,60000.0,1723.838281,57520.350305,55796.512024,1366.100248,357.738033
718,59000.0,1714.181290,57747.988720,56033.807430,1435.716456,278.464833
719,60700.0,1822.692977,58202.144301,56379.451324,1513.111761,309.581217


---

### 4. RSI

RSI(Relative Strength Index) : 주식, 선물, 옵션 등의 기술적 분석에 사용되는 보조 지표

- 현재 추세의 힘을 0~100의 퍼센트 수치로 나타낸 지표

In [8]:
def rsi(data, length):

    delta = data.diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0.0] = 0.0
    down[down > 0.0] = 0.0

    # Calculate the EWMA
    roll_up1 = up.ewm(com=(length-1), min_periods=length).mean()
    roll_down1 = down.abs().ewm(com=(length-1), min_periods=length).mean()
    
    # Calculate the RSI based on EWMA
    RSI_1 = roll_up1 / roll_down1
    RSI = 100.0 - (100.0 / (1.0 + RSI_1))
    
    df = pd.DataFrame([data,RSI]).T
    df.columns = ["close", "rsi"]

    return df

In [9]:
#Example

rsi(data['Close'],10)

,close,rsi
0,36960.0,NaN
1,36940.0,NaN
2,37480.0,NaN
3,37200.0,NaN
4,38060.0,NaN
...,...,...
727,60000.0,81.087937
728,60000.0,81.087937
729,59000.0,68.708347
730,60700.0,75.712295


---

### 5. Stochastic Oscillator

상승 추세 시장에서는 종가가 당일 최고가 근접 가격에 형성될 것이고, 반대로 하락 추세 시장에서는 종가가 당일 최저가 그접 가격에 근접한다는 것. 즉, 현재주가가 일정 기간의 주가 변동폭 중 어디에 위치하는지 백분율로 나타낸 지표

- %K = 100(C - L14)/(H14 - L14)

Where:

- C = the most recent closing price

- L = the low of the n previous trading sessions

- H = the highest price traded during the same n-day period

- %K= the current market rate for the currency pair

- %D = t-period moving average of %K

In [10]:
def stochastic(data, n=15, m=5, t=3):

    # n일중 최고가 = L
    high = data.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    low = data.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    fast_K = ((data.Close - low) / (high - low))*100
    # Fast%D (=Slow%K) 계산
    fast_D = fast_K.ewm(span=m).mean()
    # Slow%D 계산
    Slow_D = fast_D.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = pd.DataFrame([data['Close'],data['Open'],data['High'],data['Low'],fast_K,fast_D,Slow_D]).T
    df.columns = ["close",'open','high','low', "fast_K",'fast_D','slow_D']

    return df

In [11]:
stochastic(data)

,close,open,high,low,fast_K,fast_D,slow_D
0,36960.0,36580.0,37460.0,36580.0,43.181818,43.181818,43.181818
1,36940.0,37040.0,37500.0,36620.0,39.130435,40.750988,41.561265
2,37480.0,37720.0,37920.0,37020.0,67.164179,53.262500,48.247685
3,37200.0,37120.0,37420.0,36880.0,46.268657,50.357365,49.372847
4,38060.0,37200.0,38060.0,37000.0,100.000000,69.414490,59.716921
...,...,...,...,...,...,...,...
727,60000.0,59600.0,60000.0,59100.0,100.000000,90.350866,85.285514
728,60000.0,60400.0,61000.0,59900.0,84.848485,88.516739,86.901127
729,59000.0,59500.0,59600.0,58900.0,69.696970,82.243483,84.572305
730,60700.0,59100.0,60700.0,59000.0,95.454545,86.647170,85.609737


-----

### 6. Rate of Change (ROC)

최근의 가격과 n-day 전의 가격을 변화율을 측정한 것

In [12]:
def roc(data,n=5):
    N = data['Close'].diff(n)
    D = data['Close'].shift(n)
    ROC = pd.Series(N/D,name='ROC')
    
    df = pd.DataFrame([data['Close'],ROC]).T
    df.columns = ["close", "roc"]
    return df

In [13]:
roc(data)

,close,roc
0,36960.0,NaN
1,36940.0,NaN
2,37480.0,NaN
3,37200.0,NaN
4,38060.0,NaN
...,...,...
727,60000.0,0.081081
728,60000.0,0.075269
729,59000.0,0.038732
730,60700.0,0.035836


-----------------------

## 클래스로 합쳐주기 

In [14]:
class sumin():
    
    def __init__(self):
        self.data = data

    @staticmethod
    def bollinger_band(data, w=20, k=2):

        data=data['Close']
        mbb = data.rolling(w).mean()
        ubb = mbb + k * data.rolling(w).std()
        lbb = mbb - k * data.rolling(w).std()
        result = np.array([data,mbb,ubb,lbb])

        df = pd.DataFrame(result).T
        df.columns = ["close", "mbb", "ubb", "lbb"]

        return df
    
    
    @staticmethod
    def moving_average(data, w=5):
        
        data=data['Close']
        ma_w=data.ewm(span=w).mean()
        ma_5=data.ewm(span=5).mean()
        ma_10=data.ewm(span=10).mean()
        ma_15=data.ewm(span=15).mean()
        ma_20=data.ewm(span=20).mean()
        ma_30=data.ewm(span=30).mean()
        ma_60=data.ewm(span=60).mean()

        result = np.array([data,ma_w,ma_5,ma_10,ma_15,ma_20,ma_30,ma_60])

        df = pd.DataFrame(result).T
        df.columns = ["close", "user_ma_{}".format(w), "ma_5", "ma_10","ma_15","ma_20","ma_30","ma_60" ]

        if w == 5:
            del df['user_ma_5']

        return df
    
    
    @staticmethod
    def macd(data, fast=12, slow=26, signal=9) :

        data=data['Close']
        fast = data.ewm(span=fast).mean() 
        slow = data.ewm(span=slow).mean() 
        macd = fast - slow 
        signal = macd.ewm(span=signal).mean() 
        osc = macd - signal 

        result = np.array([data,macd,fast,slow, signal,osc])
        df = pd.DataFrame(result).T
        df.columns = ["close", "macd", "fast", "slow","signal","osc"]

        return df
    
    @staticmethod
    def rsi(data, length=14):

        data=data['Close']
        delta = data.diff()
        delta = delta[1:]
        up, down = delta.copy(), delta.copy()
        up[up < 0.0] = 0.0
        down[down > 0.0] = 0.0

        roll_up1 = up.ewm(com=(length-1), min_periods=length).mean()
        roll_down1 = down.abs().ewm(com=(length-1), min_periods=length).mean()

        RSI_1 = roll_up1 / roll_down1
        RSI = 100.0 - (100.0 / (1.0 + RSI_1))

        df = pd.DataFrame([data,RSI]).T
        df.columns = ["close", "rsi"]

        return df
    
    
    @staticmethod
    def stochastic(data, n=15, m=5, t=3):

        high = data.High.rolling(window=n, min_periods=1).max()
        low = data.Low.rolling(window=n, min_periods=1).min()

        fast_K = ((data.Close - low) / (high - low))*100
        fast_D = fast_K.ewm(span=m).mean()
        Slow_D = fast_D.ewm(span=t).mean()

        df = pd.DataFrame([data['Close'],data['Open'],data['High'],data['Low'],fast_K,fast_D,Slow_D]).T
        df.columns = ["close",'open','high','low', "fast_K",'fast_D','slow_D']

        return df

    @staticmethod
    def roc(data,n=5):
        
        N = data['Close'].diff(n)
        D = data['Close'].shift(n)
        ROC = pd.Series(N/D,name='ROC')

        df = pd.DataFrame([data['Close'],ROC]).T
        df.columns = ["close", "roc"]
        
        return df

In [15]:
#EX
static=sumin()

In [16]:
static.bollinger_band(data)

,close,mbb,ubb,lbb
0,36960.0,NaN,NaN,NaN
1,36940.0,NaN,NaN,NaN
2,37480.0,NaN,NaN,NaN
3,37200.0,NaN,NaN,NaN
4,38060.0,NaN,NaN,NaN
...,...,...,...,...
716,60000.0,56140.0,59371.033727,52908.966273
717,60000.0,56475.0,59852.245841,53097.754159
718,59000.0,56690.0,60138.203193,53241.796807
719,60700.0,56990.0,60740.031579,53239.968421


In [17]:
static.moving_average(data)

,close,ma_5,ma_10,ma_15,ma_20,ma_30,ma_60
0,36960.0,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000
1,36940.0,36948.000000,36949.000000,36949.333333,36949.500000,36949.666667,36949.833333
2,37480.0,37200.000000,37162.458472,37150.295858,37144.296420,37138.356164,37132.478474
3,37200.0,37200.000000,37174.826733,37165.309735,37160.377182,37155.341287,37150.212094
4,38060.0,37530.142180,37428.935903,37394.910093,37377.988286,37361.173047,37344.498887
...,...,...,...,...,...,...,...
716,60000.0,58501.075821,57388.920152,56649.522350,56055.399879,55111.799182,53105.594217
717,60000.0,59000.717214,57863.661942,57068.332057,56431.076081,55427.166977,53331.640308
718,59000.0,59000.478143,58070.268862,57309.790550,56675.735502,55657.672333,53517.488167
719,60700.0,59566.985429,58548.401796,57733.566731,57058.998787,55982.983796,53752.980358


In [18]:
static.bollinger_band(data)

,close,mbb,ubb,lbb
0,36960.0,NaN,NaN,NaN
1,36940.0,NaN,NaN,NaN
2,37480.0,NaN,NaN,NaN
3,37200.0,NaN,NaN,NaN
4,38060.0,NaN,NaN,NaN
...,...,...,...,...
716,60000.0,56140.0,59371.033727,52908.966273
717,60000.0,56475.0,59852.245841,53097.754159
718,59000.0,56690.0,60138.203193,53241.796807
719,60700.0,56990.0,60740.031579,53239.968421


In [19]:
static.bollinger_band(data)

,close,mbb,ubb,lbb
0,36960.0,NaN,NaN,NaN
1,36940.0,NaN,NaN,NaN
2,37480.0,NaN,NaN,NaN
3,37200.0,NaN,NaN,NaN
4,38060.0,NaN,NaN,NaN
...,...,...,...,...
716,60000.0,56140.0,59371.033727,52908.966273
717,60000.0,56475.0,59852.245841,53097.754159
718,59000.0,56690.0,60138.203193,53241.796807
719,60700.0,56990.0,60740.031579,53239.968421


In [20]:
static.macd(data)

,close,macd,fast,slow,signal,osc
0,36960.0,0.000000,36960.000000,36960.000000,0.000000,0.000000
1,36940.0,-0.448718,36949.166667,36949.615385,-0.249288,-0.199430
2,37480.0,16.173612,37156.351039,37140.177427,6.481409,9.692203
3,37200.0,13.228106,37170.129310,37156.901204,8.766875,4.461231
4,38060.0,45.570456,37411.903996,37366.333540,19.715109,25.855348
...,...,...,...,...,...,...
716,60000.0,1609.271920,57069.504906,55460.232986,1276.665740,332.606180
717,60000.0,1723.838281,57520.350305,55796.512024,1366.100248,357.738033
718,59000.0,1714.181290,57747.988720,56033.807430,1435.716456,278.464833
719,60700.0,1822.692977,58202.144301,56379.451324,1513.111761,309.581217


In [21]:
static.rsi(data)

,close,rsi
0,36960.0,NaN
1,36940.0,NaN
2,37480.0,NaN
3,37200.0,NaN
4,38060.0,NaN
...,...,...
727,60000.0,76.975252
728,60000.0,76.975252
729,59000.0,68.392658
730,60700.0,73.750830


In [22]:
static.stochastic(data)

,close,open,high,low,fast_K,fast_D,slow_D
0,36960.0,36580.0,37460.0,36580.0,43.181818,43.181818,43.181818
1,36940.0,37040.0,37500.0,36620.0,39.130435,40.750988,41.561265
2,37480.0,37720.0,37920.0,37020.0,67.164179,53.262500,48.247685
3,37200.0,37120.0,37420.0,36880.0,46.268657,50.357365,49.372847
4,38060.0,37200.0,38060.0,37000.0,100.000000,69.414490,59.716921
...,...,...,...,...,...,...,...
727,60000.0,59600.0,60000.0,59100.0,100.000000,90.350866,85.285514
728,60000.0,60400.0,61000.0,59900.0,84.848485,88.516739,86.901127
729,59000.0,59500.0,59600.0,58900.0,69.696970,82.243483,84.572305
730,60700.0,59100.0,60700.0,59000.0,95.454545,86.647170,85.609737


In [23]:
static.roc(data)

,close,roc
0,36960.0,NaN
1,36940.0,NaN
2,37480.0,NaN
3,37200.0,NaN
4,38060.0,NaN
...,...,...
727,60000.0,0.081081
728,60000.0,0.075269
729,59000.0,0.038732
730,60700.0,0.035836
